In [1]:
import json
import re
from functools import reduce

In [2]:
f = open('./data/cloze.ipynb', 'r')
nb = f.read()
nbd = json.loads(nb)

In [3]:
cells = [ c for c in nbd['cells'] ]
get_cells = lambda e: [c for c in cells if c['cell_type'] == e]

In [4]:
c_code = get_cells('code')

In [5]:
txt = c_code[0]['source']; txt

['%reload_ext autoreload\n', '%autoreload 2\n', '%matplotlib inline']

In [91]:
txt.append('cloze')

In [184]:
def isCloze(txtArr, cloze_marker=r'cloze'):
    present = [re.search(cloze_marker,s) for s in txtArr]
    match = None
    if len(present):
        match = reduce(lambda c, acc: c if c else acc, present)
    
    if match is not None:
        return True
    else:
        return False

isCloze(txt)

False

In [185]:
has_cloze = ['before\n', '# cloze {\n', 'code', '# }\n']
no_cloze = ['before\n', '# cl \n', 'code', '# } c\n']
no_cloze_empty = ['before\n', '# cl \n', 'code', '# } c\n', '']
has_cloze_empty = ['before\n', '# cl \n', 'code', '# } c\n', '']
empty = []

assert(isCloze(has_cloze))
assert(isCloze(no_cloze) == False)
assert(isCloze(no_cloze_empty) == False)
assert(isCloze(has_cloze_empty) == False )
assert(isCloze(empty) == False )

In [186]:
c_code[14]

{'cell_type': 'code',
 'execution_count': 43,
 'metadata': {},
 'outputs': [],
 'source': ['class DeepNN(nn.Module):\n',
  '    ### cloze {\n',
  '    def __init__(self, layers):\n',
  '        super().__init__()\n',
  '        print(layers)\n',
  '        self.layers = {}\n',
  '        for i in range(len(layers) -1):\n',
  '            print(i, [layers[i], layers[i+1]])\n',
  '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n',
  '    def forward(self, xb):\n',
  '        ab = xb\n',
  '        for i in range(len(layers) -1):\n',
  '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n',
  '        return ab\n',
  '    ### } cloze ']}

In [187]:
isCloze(c_code[14]['source'])

True

In [225]:
start_cloze_re = r'# cloze {'
end_cloze_re = r'# }'

# should return cloze code lines and the mutated source array

def extractCloze(cell_source):
    start_i = None
    end_i = None
    for i, x in enumerate(cell_source):
        if isCloze([x], start_cloze_re):
            start_i = i 
        
        if start_i is not None and isCloze([x], end_cloze_re):
            end_i = i
            
    cloze_block = cell_source[start_i+1:end_i]
    sliced_source = cell_source[:start_i+1] + cell_source[end_i:]
    
    return cloze_block, sliced_source

In [226]:
code_lines = [ f'code{i+1}\n' for i in range(10) ]
has_cloze = ['before\n', '# cloze {\n'] + code_lines  + ['# }\n']
deleted_cloze = ['before\n', '# cloze {\n'] + ['# }\n']

print(extractCloze(has_cloze)[0])
assert(extractCloze(has_cloze)[0] == code_lines)

print(extractCloze(has_cloze)[1])
assert(extractCloze(deleted_cloze)[1] == deleted_cloze)

['code1\n', 'code2\n', 'code3\n', 'code4\n', 'code5\n', 'code6\n', 'code7\n', 'code8\n', 'code9\n', 'code10\n']
['before\n', '# cloze {\n', '# }\n']


In [227]:
extractCloze(c_code[14]['source'])

(['    def __init__(self, layers):\n',
  '        super().__init__()\n',
  '        print(layers)\n',
  '        self.layers = {}\n',
  '        for i in range(len(layers) -1):\n',
  '            print(i, [layers[i], layers[i+1]])\n',
  '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n',
  '    def forward(self, xb):\n',
  '        ab = xb\n',
  '        for i in range(len(layers) -1):\n',
  '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n',
  '        return ab\n'],
 ['class DeepNN(nn.Module):\n', '    ### cloze {\n', '    ### } cloze '])

## all together now

In [244]:
get_cells = lambda cells, e: [c for c in cells if c['cell_type'] == e]

def readNotebook(path):
    # read file
    f = open(path, 'r')
    nb = f.read()
    nbd = json.loads(nb)
    
    #extract cloze
    cells = [ c for c in nbd['cells'] ]
    c_code = get_cells(cells, 'code')
    
    
    cloze_cells = [ c for c in c_code if isCloze(c['source'])] 
    cloze_blocks = [ extractCloze(c['source']) for c in cloze_cells ]
    
    return cloze_blocks, {}

In [245]:
readNotebook('./data/cloze.ipynb')

([(['    def __init__(self, layers):\n',
    '        super().__init__()\n',
    '        print(layers)\n',
    '        self.layers = {}\n',
    '        for i in range(len(layers) -1):\n',
    '            print(i, [layers[i], layers[i+1]])\n',
    '            self.layers[f"lin_{i}"] = nn.Linear(layers[i], layers[i+1])\n',
    '    def forward(self, xb):\n',
    '        ab = xb\n',
    '        for i in range(len(layers) -1):\n',
    '            ab = F.relu(self.layers[f"lin_{i}"](ab))\n',
    '        return ab\n'],
   ['class DeepNN(nn.Module):\n', '    ### cloze {\n', '    ### } cloze '])],
 {})

In [248]:
# full e2e test!
# only one cloze in 1 cell of this notebook

def get_json(fpath):
    f = open(fpath, 'r')
    file = json.loads(f.read())
    f.close()
    return file

# some test data
expected_answer = get_json('./data/cloze_answer.json')['answer']
expected_extract_nb = get_json('./data/cloze_e.ipynb')

answer, extract_nb = readNotebook('./data/cloze.ipynb')

assert(answer == expected_answer)
assert(extract_nb == expected_extract_nb)

AssertionError: 